<div align="right" style="text-align: right"><i>Peter Norvig, Feb 2020</i></div>

# CrossProduct Puzzle

The 538 Riddler [poses a type of puzzle](https://fivethirtyeight.com/features/can-you-cross-like-a-boss/) called ***CrossProduct***, which works like this:

>*Replace each "?" in the table with a single digit so that the product of the digits in each row equals the number to the right of the row, and the product of the digits in each column equals the number above the column.*


Sample puzzle:

      
|  6615 | 15552 | &nbsp; 420  |   |
|-------|-------|-------|---|
|    ?  |    ?  |    ?  |**210**|
|    ?  |    ?  |    ?  |**144**|
|    ?  |    ?  |    ?  |**54**|
|    ?  |    ?  |    ?  |**135**|
|    ?  |    ?  |    ?  |**4**|
|    ?  |    ?  |    ?  |**49**|


Solution:

|6615|15552|&nbsp; 420||
|---|---|---|---|
|7|6|5|**210**|
|9|8|2|**144**|
|3|9|2|**54**|
|5|9|3|**135**|
|1|4|1|**4**|
|7|1|7|**49**|
    






     
# Data type definitions
     
Here are the data types we will use in trying to solve CrossProduct puzzles: 
- `Digit`: a single digit, from 1 to 9 (but not 0).
- `Row`: a sequence of digits that forms a row in the table, e.g. `(7, 6, 5)`.
- `Table`: a table of digits that fill in for the "?"s; a list of rows, e.g. `[(7, 6, 5), (9, 8, 2), ...]`.
- `Products`: a list of the numbers that corresponding digits must multiply to, e.g. in the puzzle above, `[6615, 15552, 420]` for the column products, and `[210, 144, 54, 135, 4, 49]` for the row products.
- `Puzzle`: a puzzle to be solved, as defined by the row products and column products.

In [1]:
from typing import Tuple, List, Set, Iterable, Optional
from numpy  import divide, prod, transpose
from random import randint
from collections import namedtuple, Counter

Digit    = int
Row      = Tuple[Digit, ...] 
Table    = List[Row]       
Products = List[int] 
Puzzle   = namedtuple('Puzzle', 'row_prods, col_prods')

# The puzzles

Here are the puzzles given by 538 Riddler (they promised one a week for four weeks; so far we've seen three):

In [2]:
puzzles = (
    Puzzle([135,  45,  64, 280, 70],             [3000,   3969,    640]),
    Puzzle([210, 144,  54, 135,  4,  49],        [6615,  15552,    420]),
    Puzzle([280, 168, 162, 360, 60, 256, 126], [183708, 245760, 117600]))

# Filling in one row

- A first step in solving the puzzle is filling in a single row of the table.
- We will need to respect the row- and column-product constraints.
- The strategy is: for every feasible way of filling the first digit, try all ways of recursively filling the rest of the row.
- `fill_one_row(row_prod=210, col_prods=[6615, 15552, 420])` will return a set of 3-digit tuples where each tuple multiplies to 210, and each digit of the tuple evenly divides the corresponding number in `col_prods`.
  - If `col_prods` is `[]`, then there is one solution (the 0-length tuple) if `row_prod` is 1, and no solution otherwise.
  - Otherwise, try each digit `d` that divides both the `row_prod` and the first `col_prods`, and then try all ways to fill the rest of the row.

In [3]:
def fill_one_row(row_prod, col_prods) -> Set[Row]:
    "All permutations of digits that multiply to `row_prod` and evenly divide `col_prods`."
    return ({()}   if not col_prods and row_prod == 1 else
            set()  if not col_prods and row_prod != 1 else
            {(d, *rest) for d in range(1, 10)
             if (row_prod / d).is_integer() and (col_prods[0] / d).is_integer()
             for rest in fill_one_row(row_prod // d, col_prods[1:])})

Some examples:

In [4]:
fill_one_row(210, [6615, 15552, 420]) # There are two ways to fill the first row

{(5, 6, 7), (7, 6, 5)}

In [5]:
fill_one_row(729, [90, 90, 90])

{(9, 9, 9)}

In [6]:
fill_one_row(729, [90, 90, 90, 90])

{(1, 9, 9, 9),
 (3, 3, 9, 9),
 (3, 9, 3, 9),
 (3, 9, 9, 3),
 (9, 1, 9, 9),
 (9, 3, 3, 9),
 (9, 3, 9, 3),
 (9, 9, 1, 9),
 (9, 9, 3, 3),
 (9, 9, 9, 1)}

In [7]:
fill_one_row(729, [30, 90, 90, 90])

{(1, 9, 9, 9), (3, 3, 9, 9), (3, 9, 3, 9), (3, 9, 9, 3)}

In [8]:
fill_one_row(7**8, [7]*9)

{(1, 7, 7, 7, 7, 7, 7, 7, 7),
 (7, 1, 7, 7, 7, 7, 7, 7, 7),
 (7, 7, 1, 7, 7, 7, 7, 7, 7),
 (7, 7, 7, 1, 7, 7, 7, 7, 7),
 (7, 7, 7, 7, 1, 7, 7, 7, 7),
 (7, 7, 7, 7, 7, 1, 7, 7, 7),
 (7, 7, 7, 7, 7, 7, 1, 7, 7),
 (7, 7, 7, 7, 7, 7, 7, 1, 7),
 (7, 7, 7, 7, 7, 7, 7, 7, 1)}

# Solving a whole puzzle

- We can solve a whole puzzle with a similar strategy.
- For every possible way of filling the first row,  try every way of recursively solving the rest of the puzzle. 
- `solve` finds the first solution to a puzzle. (A well-formed puzzle has exactly one solution, but some might have more or less.)
- `solutions` yields all possible solutions to a puzzle. There are three main cases to consider:
  - A puzzle with no rows has the empty table, `[]`, as a solution, as long as the column products are all 1.
  - A puzzle with rows might have solutions, as long as the column products are all integers. Call `fill_row` to get all possible ways to fill the first row, and for each one recursively call `solutions` to get all the possible ways of filling the rest of the rows (making sure to pass in an altered `col_prods` where each element is divided by the corresponding element in the first row).
  - Otherwise there are no solutions.

In [9]:
def solve(puzzle) -> Optional[Table]: return next(solutions(puzzle), None)

def solutions(puzzle) -> Iterable[Table]:
    """Yield all tables that solve the puzzle.
    The product of the digits in row r must equal row_prods[r], for all r.
    The product of the digits in column c must equal col_prods[c], for all c."""
    row_prods, col_prods = puzzle
    if not row_prods and all(c == 1 for c in col_prods):
        yield []
    elif row_prods and all(c == int(c) for c in col_prods):
        yield from ([row1, *rows]
                    for row1 in fill_one_row(row_prods[0], col_prods)
                    for rows in solutions(Puzzle(row_prods[1:], list(divide(col_prods, row1)))))

# Solutions

Here are  solutions to the puzzles posed by *The Riddler*:

In [10]:
[solve(p) for p in puzzles]

[[(3, 9, 5), (5, 9, 1), (8, 1, 8), (5, 7, 8), (5, 7, 2)],
 [(7, 6, 5), (9, 8, 2), (3, 9, 2), (5, 9, 3), (1, 4, 1), (7, 1, 7)],
 [(7, 8, 5), (3, 8, 7), (9, 6, 3), (9, 8, 5), (3, 5, 4), (4, 8, 8), (9, 2, 7)]]

# Prettier solutions

Those are the correct solutions. However, we could make them look nicer:

In [11]:
from IPython.display import Markdown

def pretty(puzzle, table=None) -> str:
    """A puzzle and the filled-in table as a str that will be pretty in Markdown."""
    row_prods, col_prods = puzzle
    table = table or solve(puzzle)
    head  = surround(col_prods + [''])
    dash  = surround(['---'] * (1 + len(col_prods)))
    rest  = [surround(row + (f'**{rp}**',))
             for row, rp in zip(table, row_prods)]
    return '\n'.join([head, dash, *rest])

def surround(items, delim='|') -> str: 
    """Like str.join, but delimiter is outside items as well as between."""
    return delim + delim.join(map(str, items)) + delim

In [12]:
Markdown('\n\n'.join(map(pretty, puzzles)))

|3000|3969|640||
|---|---|---|---|
|3|9|5|**135**|
|5|9|1|**45**|
|8|1|8|**64**|
|5|7|8|**280**|
|5|7|2|**70**|

|6615|15552|420||
|---|---|---|---|
|7|6|5|**210**|
|9|8|2|**144**|
|3|9|2|**54**|
|5|9|3|**135**|
|1|4|1|**4**|
|7|1|7|**49**|

|183708|245760|117600||
|---|---|---|---|
|7|8|5|**280**|
|3|8|7|**168**|
|9|6|3|**162**|
|9|8|5|**360**|
|3|5|4|**60**|
|4|8|8|**256**|
|9|2|7|**126**|

# Making new puzzles

Can we make new puzzles? Can we make well-formed ones (those with exactly one solution)? Here is an approach:
- Make a table filled with random digits (`random_table`).
- Make a puzzle from the row and column products of the table (`table_puzzle`).
- Repeat `N` times (`random_puzzles`).
- Optionally, check if puzzles are `well-formed`.


In [13]:
def random_table(nrows, ncols) -> Table:
    "Make a table of random digits of the given size."
    return [tuple(randint(1, 9) for c in range(ncols))
            for r in range(nrows)]

def table_puzzle(table) -> Puzzle:
    "Given a table, compute the puzzle it is a solution for."
    return Puzzle([prod(row) for row in table], 
                  [prod(col) for col in transpose(table)])

def random_puzzles(N, nrows, ncols) -> List[Puzzle]: 
    "Return a list of `N` random puzzles."
    return [table_puzzle(random_table(nrows, ncols)) for _ in range(N)]

def well_formed(puzzle) -> bool: 
    "Does the puzzle have exactly one solution?"
    S = solutions(puzzle)
    first, second = next(S, None), next(S, None)
    return first is not None and second is None

In [14]:
random_table(nrows=2, ncols=3)

[(5, 3, 8), (7, 2, 1)]

In [15]:
table_puzzle(_)

Puzzle(row_prods=[120, 14], col_prods=[35, 6, 8])

In [16]:
well_formed(_)

False

How likely are random puzzles (of various sizes) to be well-formed?

In [17]:
N = 300
for r, c in [(3, 3), (3, 4), (4, 3), (3, 5), (5, 3), (4, 4), (6, 3), (5, 4)]:
    w = sum(map(well_formed, random_puzzles(N, r, c))) / N
    print(f'{w:3.0%} of random puzzles with {r} rows and {c} cols ({r * c:2} cells) are well-formed')

38% of random puzzles with 3 rows and 3 cols ( 9 cells) are well-formed
15% of random puzzles with 3 rows and 4 cols (12 cells) are well-formed
14% of random puzzles with 4 rows and 3 cols (12 cells) are well-formed
 6% of random puzzles with 3 rows and 5 cols (15 cells) are well-formed
 5% of random puzzles with 5 rows and 3 cols (15 cells) are well-formed
 3% of random puzzles with 4 rows and 4 cols (16 cells) are well-formed
 1% of random puzzles with 6 rows and 3 cols (18 cells) are well-formed
 0% of random puzzles with 5 rows and 4 cols (20 cells) are well-formed


We see that most puzzles are not well-formed. Smaller sizes are more likely to yield well-formed puzzles.

# Speed

How fast is it to solve a random puzzle? We can do a hundred small (5x3) puzzles in under a second:

In [18]:
%time len([solve(p) for p in random_puzzles(100, 5, 3)])

CPU times: user 179 ms, sys: 7.47 ms, total: 187 ms
Wall time: 181 ms


100

Puzzles that are even a little bit larger can be a lot slower. For example, a 10 x 4 puzzle can take from 5 milliseconds to 5 seconds or so:

In [19]:
puzz = random_puzzles(1, 10, 4)[0]
%time Markdown(pretty(puzz))

CPU times: user 5.94 ms, sys: 237 µs, total: 6.18 ms
Wall time: 6 ms


|645120|201600|1382400|1290240||
|---|---|---|---|---|
|2|3|5|5|**150**|
|3|4|4|4|**192**|
|6|5|2|3|**180**|
|8|4|4|8|**1024**|
|8|4|4|8|**1024**|
|1|5|2|4|**40**|
|1|2|9|6|**108**|
|5|1|3|1|**15**|
|7|3|8|7|**1176**|
|8|7|5|2|**560**|

What could we do to speed that up?
- We could treat it as a constraint satisfaction problem (CSP), and use a highly-optimized [CSP solver](https://developers.google.com/optimization/cp/cp_solver).
- We could use some of the lessons that CSP solvers use, such as trying the most constrained variables first. If a row- (or column-) product is divisible by 5 (or 7), then one of the digits in the row (or column) must be 5 (or 7). (That's not true of any other digit. For example, if a product is divisible by 2 and 3, it might be that the digits 2 and 3 appear, but perhaps 6 appears instead.) So instead of filling in the table row by row, top to bottom, try filling in the 5s and 7s first, then fill other rows that have a small number of possible fillers.

# Tests

A suite of unit tests:

In [20]:
def test():
    "Test suite for CrossProduct functions."
    assert fill_one_row(1, [])  == {()}
    assert fill_one_row(2, [])  == set()
    assert fill_one_row(9, [9]) == {(9,)}
    assert fill_one_row(10, [10])  == set()
    assert fill_one_row(73, [360, 360, 360])  == set()
    
    assert solve(Puzzle([], []))   == []
    assert solve(Puzzle([], [1]))  == []
    assert solve(Puzzle([], [2]))  == None
    assert solve(Puzzle([5], [5])) == [(5,)]
    assert solve(Puzzle([0], [0])) == None # Maybe should allow zero as a digit?
    assert solve(Puzzle([2, 12], [3, 8])) == [(1, 2), (3, 4)]

    assert fill_one_row(729, [90, 126, 81]) == {(9, 9, 9)} # Unique fill
    
    assert fill_one_row(729, [90, 126, 81, 30]) == {
     (3, 9, 9, 3), (9, 3, 9, 3), (9, 9, 3, 3), (9, 9, 9, 1)}
    
    # 72 has the most ways to fill a 3-digit row
    assert max(range(1, 100), key=lambda n: len(fill_one_row(n, [5*7*8*9]*3))) == 72
    assert fill_one_row(72, [72, 72, 72])  == { 
        (1, 8, 9),
        (1, 9, 8),
        (2, 4, 9),
        (2, 6, 6),
        (2, 9, 4),
        (3, 3, 8),
        (3, 4, 6),
        (3, 6, 4),
        (3, 8, 3),
        (4, 2, 9),
        (4, 3, 6),
        (4, 6, 3),
        (4, 9, 2),
        (6, 2, 6),
        (6, 3, 4),
        (6, 4, 3),
        (6, 6, 2),
        (8, 1, 9),
        (8, 3, 3),
        (8, 9, 1),
        (9, 1, 8),
        (9, 2, 4),
        (9, 4, 2),
        (9, 8, 1)}
    
    assert solve(Puzzle([210, 144, 54, 135, 4, 49], [6615, 15552, 420])) == [
        (7, 6, 5), 
        (9, 8, 2), 
        (3, 9, 2), 
        (5, 9, 3), 
        (1, 4, 1), 
        (7, 1, 7)]
    
    assert sorted(solutions(Puzzle([8, 8, 1], [8, 8, 1]))) == [ # Multi-solution puzzle
        [(1, 8, 1), 
         (8, 1, 1), 
         (1, 1, 1)],
        [(2, 4, 1), 
         (4, 2, 1), 
         (1, 1, 1)],
        [(4, 2, 1), 
         (2, 4, 1), 
         (1, 1, 1)],
        [(8, 1, 1), 
         (1, 8, 1), 
         (1, 1, 1)]]
    
    assert not list(solutions(Puzzle([8, 8, 1], [8, 8, 2]))) # Unsolvable puzzle
    
    assert solve(Puzzle([1470, 720, 270, 945, 12, 343], 
                        [6615, 15552, 420, 25725])) == [ # 4 column puzzle
        (7, 6, 5, 7),
        (9, 8, 2, 5),
        (3, 9, 2, 5),
        (5, 9, 3, 7),
        (1, 4, 1, 3),
        (7, 1, 7, 7)]
    
    puzz  = Puzzle([6, 120, 504], [28, 80, 162])
    table = [(1, 2, 3), 
             (4, 5, 6), 
             (7, 8, 9)]
    assert solve(puzz) == table
    assert table_puzzle(table) == puzz
    assert well_formed(puzz)
    
    assert not well_formed(Puzzle([7, 7], [7, 7]))
    assert well_formed(Puzzle([64, 224, 189, 270, 405, 144, 105], 
                              [308700, 12960, 1119744]))
    
    assert surround((1, 2, 3)) == '|1|2|3|'
    
    return True
    
test()

True